## Problem 2

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t
from scipy.integrate import quad

In [4]:
def ew_cov_corr(df, lmbd, func='cov'):
    df -= df.mean(axis=0)
    m, n = df.shape
    wts = np.empty(m)
    for i in range(m):
        wts[i] = (1 - lmbd) * lmbd ** (m - i - 1)
        
    wts /= np.sum(wts)
    wts = wts.reshape(-1, 1)
    if func == 'cov':   
        res = (wts * df).T @ df
        
    elif func == 'corr':
        res = (wts * df).T @ df
        std_devs = np.sqrt(np.diag(res))
        res /= np.outer(std_devs, std_devs)
        
    return res


def var_ew(data, lmbd, alpha=0.05):
    mu = data.mean()
    ew_sigma2 = ew_cov_corr(data, lmbd, 'cov')
    VaR = -norm.ppf(alpha, mu, np.sqrt(ew_sigma2))[0, 0]
    
    VaR_diff = VaR + mu
    VaR_diff = VaR_diff[0]

    return pd.DataFrame({"VaR Absolute": [VaR], 
                         "VaR Diff from Mean": [VaR_diff]})

def es_ew(data, lmbd, alpha=0.05):
    mu = data.mean()
    
    std = np.sqrt(ew_cov_corr(data, lmbd, 'cov')).iloc[0, 0]
    data += mu
    
    res = var_ew(data, lmbd, alpha)
    VaR = res.iloc[0, 0]
    
    def integrand(x, mu, std):
        return x * norm.pdf(x, loc=mu, scale=std)
    
    ES, _ = quad(lambda x: integrand(x, mu, std), -np.inf, -VaR)
    ES /= -alpha
    
    ES_diff = ES + mu
    ES_diff = ES_diff[0]
    
    return pd.DataFrame({"ES Absolute": [ES], 
                         "ES Diff from Mean": [ES_diff]})

In [5]:
df = pd.read_csv('problem1.csv')
var_ew(df, 0.97, 0.05)

,VaR Absolute,VaR Diff from Mean
0,0.091169,0.09029


In [6]:
df = pd.read_csv('problem1.csv')
es_ew(df, 0.97, 0.05)

,ES Absolute,ES Diff from Mean
0,0.114107,0.113227


In [7]:
def fit_general_t(data):
    nu, mu, sigma = t.fit(data)
    return mu, sigma, nu

def var_t(data, alpha=0.05):
    mu, sigma, nu = fit_general_t(data)   
    VaR = -t.ppf(alpha, nu, mu, sigma)
    VaR_diff = VaR + mu
    
    return pd.DataFrame({"VaR Absolute": [VaR], 
                         "VaR Diff from Mean": [VaR_diff]})

def es_t(data, alpha=0.05):
    mu, sigma, nu = fit_general_t(data)
    res = var_t(data, alpha)
    VaR = res.iloc[0, 0]
    
    def integrand(x, mu, sigma, nu):
        return x * t.pdf(x, df=nu, loc=mu, scale=sigma)
    
    ES, _ = quad(lambda x: integrand(x, mu, sigma, nu), -np.inf, -VaR)
    ES /= -alpha
    ES_diff = ES + mu
    
    return pd.DataFrame({"ES Absolute": [ES], 
                         "ES Diff from Mean": [ES_diff]})

In [10]:
df = pd.read_csv('problem1.csv')
var_t(df, 0.05)

,VaR Absolute,VaR Diff from Mean
0,0.076476,0.076382


In [11]:
df = pd.read_csv('problem1.csv')
es_t(df, 0.05)

,ES Absolute,ES Diff from Mean
0,0.113218,0.113124


In [12]:
def historic(data, N=100000, alpha=0.05):
    np.random.seed(50)
    simulated_draws = np.random.choice(data.iloc[:,0], size=N, replace=True)
    mu = simulated_draws.mean()
    
    sorted_data = np.sort(simulated_draws)
    
    VaR_index = int(len(sorted_data) * 0.05)
    VaR = -sorted_data[VaR_index]

    VaR_diff = VaR + mu
    
    ES = -sorted_data[sorted_data <= -VaR].mean()
    ES_diff = ES + mu
    
    
    return pd.DataFrame({"VaR Absolute": [VaR], 
                         "VaR Diff from Mean": [VaR_diff],
                         "ES Absolute": [ES], 
                         "ES Diff from Mean": [ES_diff]})

In [13]:
df = pd.read_csv('problem1.csv')
historic(df)

,VaR Absolute,VaR Diff from Mean,ES Absolute,ES Diff from Mean
0,0.075862,0.074978,0.115348,0.114465


## Problem 3

In [34]:
from MyLib.calc_returns import return_calculate
from MyLib.copula import simulateCopula

In [52]:
prices = pd.read_csv('DailyPrices.csv')
returns = return_calculate(prices)
returns -= returns.mean(numeric_only=True)

returns = returns.copy()
returns = returns.drop('Date', axis=1)
returns.head()

,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,META,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
0,-0.010992,-0.014177,-0.017632,-0.002763,-0.025527,-0.018099,-0.025331,-0.017797,-0.017836,-0.033266,...,-0.001401,-0.012419,0.030937,-0.011172,-0.018637,-0.006037,-0.015836,0.002191,-0.013291,-0.024982
1,-0.004221,-0.008781,-0.011940,-0.011317,-0.018055,-0.010519,0.015325,-0.011924,-0.004872,-0.013890,...,-0.004862,0.006623,-0.016289,-0.000172,0.002453,-0.009821,-0.001483,-0.012350,-0.003810,0.014647
2,0.017517,0.008688,0.018146,0.026385,0.014076,0.023841,0.033561,0.027030,0.015107,0.008882,...,0.016431,0.018469,-0.010200,0.037793,0.019336,0.010477,0.022107,0.008847,0.025901,0.022302
3,0.006088,-0.010184,0.000700,0.002288,0.015407,-0.010652,0.019343,-0.010477,0.007202,0.007625,...,0.002228,0.012119,0.027781,0.015137,0.019688,0.014773,-0.012499,0.026460,0.028808,0.009735
4,0.015087,0.018274,0.022011,0.026238,0.023659,0.020069,0.035767,0.020686,0.007594,0.040962,...,0.007106,0.015418,0.036603,-0.004923,0.019468,0.024658,0.036130,0.015775,0.004292,0.001067


In [53]:
portfolio = pd.read_csv('portfolio.csv')
for stock in portfolio["Stock"]:
    portfolio.loc[portfolio['Stock'] == stock, 'Starting Price'] = prices.iloc[-1][stock]

portfolio.loc[portfolio['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio.loc[portfolio['Portfolio'] == 'C', 'Distribution'] = 'Normal'
print(portfolio.head())

  Portfolio  Stock  Holding  Starting Price Distribution
0         A   AAPL      158      174.789993            T
1         A   MSFT      178      317.010010            T
2         A   AMZN      110      129.119995            T
3         A   NVDA       54      416.100006            T
4         A  GOOGL       69      130.250000            T


In [62]:
portfolio_A = portfolio[portfolio['Portfolio'] == 'A'].copy()
portfolio_A = portfolio_A.drop('Portfolio', axis=1)
portfolio_B = portfolio[portfolio['Portfolio'] == 'B'].copy()
portfolio_B = portfolio_B.drop('Portfolio', axis=1)
portfolio_C = portfolio[portfolio['Portfolio'] == 'C'].copy()
portfolio_C = portfolio_C.drop('Portfolio', axis=1)

stocks_in_A = portfolio_A['Stock'].tolist()
returns_A = returns[stocks_in_A]
stocks_in_B = portfolio_B['Stock'].tolist()
returns_B = returns[stocks_in_B]
stocks_in_C = portfolio_C['Stock'].tolist()
returns_C = returns[stocks_in_C]

print(portfolio_A.shape)
print(returns_A.shape)

(33, 4)
(265, 33)


In [64]:
risk_A = simulateCopula(portfolio_A, returns_A)
print('Portfolio A:')
print(risk_A)
risk_B = simulateCopula(portfolio_B, returns_B)
print('Portfolio B:')
print(risk_B)
risk_C = simulateCopula(portfolio_C, returns_C)
print('Portfolio C:')
print(risk_C)

Portfolio A:
    Stock         VaR95          ES95 VaR95_Pct  ES95_Pct
0    AAPL    792.713124   1181.931658  0.028704  0.042798
1    MSFT   1803.175697   2593.308164  0.031955  0.045958
2    AMZN     568.75292     804.57343  0.040044  0.056647
3    NVDA   1227.382946   1816.302882  0.054625  0.080835
4   GOOGL    312.537014    446.887381  0.034776  0.049725
5    TSLA    917.710206   1259.783238   0.06246  0.085742
6    GOOG    764.421581   1095.374824  0.034875  0.049974
7   BRK-B   1241.494663   1779.169354  0.018633  0.026702
8    META   1142.776572   1926.806952  0.045488  0.076696
9     UNH    679.367697   1002.005397  0.022006  0.032457
10    XOM    215.176229    290.745726  0.027531  0.037199
11    LLY   2710.479492   4140.870154   0.02474  0.037796
12    JPM     546.26526     829.77498    0.0245  0.037215
13    JNJ    192.400409     293.27496  0.015369  0.023426
14      V    268.252625    394.148696  0.020377   0.02994
15     PG    179.024518    246.732755  0.016179  0.022298
1